# ◆[Backtest](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Strategy.buy) with OANDA API ver.H4
###### Create Date：2020/02/08　Author：M.Hasegawa
### ────────────────────────────────────────────────────────────────

#### 【table of contents】
- 0.[**Import module**](#Import_module)
- 1.[**Import data**](#Import_data)
- 2.[**Strategy**](#Strategy)
- 3.[**Backtest**](#Backtest)
- 4.[**Optimisation**](#Optimisation)

- ref:https://kernc.github.io/backtesting.py/doc/examples/Multiple%20Time%20Frames.html
- ref:https://saidataisei.hatenablog.com/entry/2019/10/13/003622
- ref:http://mmorley.hatenablog.com/entry/fx_backtesting01

## 0. Import module<a id='Import_module'></a>
- pip install backtesting
- pip install mpl_finance
- pip install oandapyV20
- pip install git+https://github.com/oanda/oandapy.git
- pip install TA_Lib-0.4.17-cp37-cp37m-win_amd64.whl

###### TA-LIB
- ref: https://qiita.com/ConnieWild/items/cb50f36425a683c914d2
- ref: https://www.lfd.uci.edu/~gohlke/pythonlibs/#ta-lib

In [1]:
%matplotlib inline
import oandapy
import pytz
import configparser
import pandas as pd
import numpy  as np
import talib as ta

from datetime         import datetime
from oandapyV20       import API
from backtesting      import Backtest, Strategy
from backtesting.lib  import crossover
from backtesting.lib  import resample_apply
from backtesting.test import SMA, GOOG
from pandas.core      import resample

config = configparser.ConfigParser()
config.read('./config_OANDA.txt')
account_id = config['oanda']['account_id']
api_key = config['oanda']['api_key']

import warnings
warnings.filterwarnings("ignore")

# ============================================================================
# Conv Japan Time
# ============================================================================
def iso_to_jp(iso):
    date = None
    try:
        date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%fZ')
        date = pytz.utc.localize(date).astimezone(pytz.timezone("Asia/Tokyo"))
    except ValueError:
        try:
            date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%f%z')
            date = date.astimezone(pytz.timezone("Asia/Tokyo"))
        except ValueError:
            pass
    return date

# ============================================================================
# Conv Format
# ============================================================================
def date_to_str(date):
    if date is None:
        return ''
    return date.strftime('%Y/%m/%d %H:%M:%S')

# ============================================================================
# SMA
# ============================================================================
def SMA(array, n):
    return pd.Series(array).rolling(n).mean()

# ============================================================================
# RSI
# ============================================================================
def RSI(array, n):
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

# ============================================================================
# RCI:http://lowcost-greatidea.jp/technicalanalysis/rci/
# ============================================================================
def RCI(close, period):
    rank_period = np.arange(period, 0, -1)
    length = len(close)
    rci = np.zeros(length)
    
    for i in range(length):
        if i < period - 1:
            rci[i] = 0
        else :
            rank_price = pd.Series(close)[i - period + 1: i + 1].rank(method='min', ascending = False).values
            rci[i] = np.int32((1 - (6 * sum((rank_period - rank_price)**2)) / (period**3 - period)) * 100)
            
    return rci

# ============================================================================
# BB2
# ============================================================================
def bb2(array, n):
    gain=pd.DataFrame(array)
    gain.columns=['close']
    upper2, middle, lower2 = ta.BBANDS(gain.close, n,2,2,0)
    gain['bb_upper'] = upper2
    gain['bb_lower'] = lower2
    return gain['bb_upper'],gain['bb_lower']

# ============================================================================
# BB3
# ============================================================================
def bb3(array, n):
    gain=pd.DataFrame(array)
    gain.columns=['close']
    upper3, middle, lower3 = ta.BBANDS(gain.close, n,3,3,0)
    gain['bb_upper'] = upper3
    gain['bb_lower'] = lower3
    return gain['bb_upper'],gain['bb_lower']

# ============================================================================
# ADX
# ============================================================================
def adx(array,n):
    gain=pd.DataFrame(array)
    gain=gain.T
    gain.columns=['close','high','low']
    gain['adx'] = ta.ADX(gain['high'],gain['low'],gain['close'],n)
    return gain['adx']

# ============================================================================
# ヒストリカルデータ取得
# ============================================================================
def df_init(data):
    
    df = pd.DataFrame(data["candles"])

    df['time'] = df['time'].apply(lambda x: iso_to_jp(x))   # 日本時間に変換
    df['time'] = df['time'].apply(lambda x: date_to_str(x)) # 形式変換（文字列型）
    df["time"] = pd.to_datetime(df["time"])                  # 型変換
    #df["time"] = df["time"].apply(mdates.date2num)          # 数値変換

    df = df[["time","openAsk","highAsk","lowAsk","closeAsk"]].copy()
    df = df.rename(columns={"openAsk":"Open","highAsk":"High","lowAsk":"Low","closeAsk":"Close"})
    df = df.set_index("time")

    #print('df.shape=',df.shape)
    #display(df.head(2))
    #display(df.tail(2))
    
    return df

C:\Users\arsma\Anaconda3\lib\site-packages\backtesting\_plotting.py:37: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# 1. Import data<a id='Import_data'></a>

URL:http://developer.oanda.com/rest-live/rates/

- M30・・・30 minute candlesticks, hour alignment
- H1・・・1 hour candlesticks, hour alignment
- H4・・・4 hour candlesticks, day alignment
- D・・・1 day candlesticks, day alignment
- W・・・1 week candlesticks, aligned to start of week
- M・・・1 month candlesticks, aligned to first day of the month

In [2]:
# API呼出
oa = oandapy.API(environment="live", access_token=api_key)

# 過去約1年分
data = oa.get_history(instrument="USD_JPY", granularity='H4', count=1050) 
df_1050 = df_init(data)

# 過去約2年分
data = oa.get_history(instrument="USD_JPY", granularity='H4', count=2100)
df_2100 = df_init(data)

# 過去約3年分
data = oa.get_history(instrument="USD_JPY", granularity='H4', count=3150)
df_3150 = df_init(data)

# 2. Strategy<a id='Strategy'></a>

## 2-1. Best of RCIStrategy

In [3]:
class RCIStrategy(Strategy):

    n1 = 21; n2 = 75
    rci_s = 9; rci_m = 24; rci_l = 52
    
    def init(self):
        # SMA
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        # BB
        self.bb_u2,self.bb_l2 = self.I(bb2,self.data.Close,self.n1)
        self.bb_u3,self.bb_l3 = self.I(bb3,self.data.Close,self.n1)
        # RCI
        self.rci_s = self.I(RCI, self.data.Close, self.rci_s)
        self.rci_m = self.I(RCI, self.data.Close, self.rci_m)
        self.rci_l = self.I(RCI, self.data.Close, self.rci_l)
        
    def next(self):
        price = self.data.Close[-1] # 前回の終値      
        
        # *******************************************************************************
        # ■ ロング
        # *******************************************************************************
        if (not self.position and ((1==0) 
             # ▼ロングのEntryルール(理想は3重底)
             or (self.rci_s[-1] < -95)
             or (self.rci_s[-1] < -82 and self.rci_m[-1] < 10)
             or (self.rci_s[-1] < -50 and self.rci_m[-1] < -40 and self.rci_l[-1] < 50)
             or (self.rci_s[-1] < -70 and self.rci_m[-1] < 20 and self.rci_l[-1] < -60)
            )):
            self.buy(sl=.98 * price)
            
        elif (self.position.is_short and ((1==0) 
               # ▼ロングのEntryルール(ショートからの切替)
               or (self.rci_s[-1] < -87 and self.rci_m[-1] < 100 and self.rci_l[-1] < 100) 
             )):
            self.buy(sl=.98 * price)
            
        elif (self.position.is_long and ((1==0) 
               # ▼ロングのExitルール(理想は3天井)
               or (self.rci_s[-1] > 100)
               or (self.rci_m[-1] > 88) 
               or (self.rci_l[-1] > 75)
               or (self.rci_s[-1] > 60 and self.rci_m[-1] > -90 and self.rci_l[-1] > 25)
               or (self.rci_s[-1] > 95 and self.rci_m[-1] > 72 and self.rci_l[-1] > -50)
              )):
            self.position.close() 
            
        # *******************************************************************************
        # ■ ショート
        # *******************************************************************************
        if (not self.position and ((1==0) 
             # ▼ショートのEntryルール(理想は3天井)
             or (self.rci_m[-1] > 95)
             or (self.rci_s[-1] > 70 and self.rci_m[-1] > 60 and self.rci_l[-1] > -100)
             or (self.rci_s[-1] > 70 and self.rci_m[-1] > -100 and self.rci_l[-1] > 60)
            )):
            self.sell(sl=1.02 * price)
            
        elif (self.position.is_long and ((1==0) 
               # ▼ショートのEntryルール(ロングからの切替)
               or (self.rci_s[-1] > 95 and self.rci_m[-1] > 50 and self.rci_l[-1] > -50) 
             )):
            self.sell(sl=1.02 * price)
            
        elif (self.position.is_short and ((1==0) 
               # ▼ショートのExitルール(理想は3重底)
               or (self.rci_s[-1] < -91)
               or (self.rci_m[-1] < -86)
               or (self.rci_l[-1] < -76)
               or (self.rci_s[-1] < -75 and self.rci_m[-1] < -75 and self.rci_l[-1] < 0)
               or (self.rci_s[-1] < -70 and self.rci_m[-1] < -50 and self.rci_l[-1] < -20)
              )):
            self.position.close()
            

## 2-2. Testing of RCIStrategy

In [4]:
class RCIStrategy2(Strategy):

    n1 = 21; n2 = 75
    rci_s = 9; rci_m = 24; rci_l = 52
    
    def init(self):
        # SMA
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        # BB
        self.bb_u2,self.bb_l2 = self.I(bb2,self.data.Close,self.n1)
        self.bb_u3,self.bb_l3 = self.I(bb3,self.data.Close,self.n1)
        # RCI
        self.rci_s = self.I(RCI, self.data.Close, self.rci_s)
        self.rci_m = self.I(RCI, self.data.Close, self.rci_m)
        self.rci_l = self.I(RCI, self.data.Close, self.rci_l)
        
    def next(self):
        price = self.data.Close[-1] # 前回の終値      
        
        # *******************************************************************************
        # ■ ロング
        # *******************************************************************************
        if (not self.position and ((1==0) 
             # ▼ロングのEntryルール(理想は3重底)
             or (self.rci_s[-1] < -95)
             or (self.rci_s[-1] < -82 and self.rci_m[-1] < 10)
             or (self.rci_s[-1] < -50 and self.rci_m[-1] < -40 and self.rci_l[-1] < 50)
             or (self.rci_s[-1] < -70 and self.rci_m[-1] < 20 and self.rci_l[-1] < -60)
            )):
            self.buy(sl=.98 * price)
            
        elif (self.position.is_short and ((1==0) 
               # ▼ロングのEntryルール(ショートからの切替)
               or (self.rci_s[-1] < -87 and self.rci_m[-1] < 100 and self.rci_l[-1] < 100) 
             )):
            self.buy(sl=.98 * price)
            
        elif (self.position.is_long and ((1==0) 
               # ▼ロングのExitルール(理想は3天井)
               or (self.rci_s[-1] > 100)
               or (self.rci_m[-1] > 88) 
               or (self.rci_l[-1] > 75)
               or (self.rci_s[-1] > 60 and self.rci_m[-1] > -90 and self.rci_l[-1] > 25)
               or (self.rci_s[-1] > 95 and self.rci_m[-1] > 72 and self.rci_l[-1] > -50)
              )):
            self.position.close() 
            
        # *******************************************************************************
        # ■ ショート
        # *******************************************************************************
        if (not self.position and ((1==0) 
             # ▼ショートのEntryルール(理想は3天井)
             or (self.rci_m[-1] > 95)
             or (self.rci_s[-1] > 70 and self.rci_m[-1] > 60 and self.rci_l[-1] > -100)
             or (self.rci_s[-1] > 70 and self.rci_m[-1] > -100 and self.rci_l[-1] > 60)
            )):
            self.sell(sl=1.02 * price)
            
        elif (self.position.is_long and ((1==0) 
               # ▼ショートのEntryルール(ロングからの切替)
               or (self.rci_s[-1] > 95 and self.rci_m[-1] > 50 and self.rci_l[-1] > -50) 
             )):
            self.sell(sl=1.02 * price)
            
        elif (self.position.is_short and ((1==0) 
               # ▼ショートのExitルール(理想は3重底)
               or (self.rci_s[-1] < -91)
               or (self.rci_m[-1] < -86)
               or (self.rci_l[-1] < -76)
               or (self.rci_s[-1] < -75 and self.rci_m[-1] < -75 and self.rci_l[-1] < 0)
               or (self.rci_s[-1] < -70 and self.rci_m[-1] < -50 and self.rci_l[-1] < -20)
              )):
            self.position.close()
            

# 3. Backtest<a id='Backtest'></a>
- [backtesting](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Strategy.buy)

#### H4×1050（約1年間）Duration= 245 days 05:00:00（2020/2/12実施）
- Equity Final($)= 1171.76
- Trades= 35
- Win Rate(%)= 91.43
- Return(%)= 17.18
- Avg. Trade Duration= 5 days 17:00:00
- SQN= 5.33

#### H4×2100（約2年間）Duration= 494 days 05:00:00（2020/2/12実施）
- Equity Final($)= 1277.69
- Trades= 67
- Win Rate(%)= 88.06
- Return(%)= 27.77
- Avg. Trade Duration= 6 days 06:00:00
- SQN= 4.38

#### H4×3150（約3年間）Duration= 739 days 04:00:00（2020/2/12実施）
- Equity Final($)= 1363.79
- Trades= 109
- Win Rate(%)= 83.49
- Return(%)= 36.38
- Avg. Trade Duration= 5 days 20:00:00
- SQN= 4.59

In [11]:
bt = Backtest(df_1050, # df_1050,df_2100,df_3150
              RCIStrategy2,
              cash=1000,  # 所持金1000ドル(=約10万)
              commission=0.0002, # 取引手数料（為替価格に対する倍率で指定、為替価格100円でcommission=0.0002なら0.02円）
              trade_on_close=True # True：現在の終値に関してエントリー／ False：次の始値にエントリー
             )
out=bt.run()

#print('Duration=',out['Duration'])
print('Equity Final($)=',round(out['Equity Final [$]'],2))
print('Trades=',out['# Trades'])
print('Win Rate(%)=',round(out['Win Rate [%]'],2))
print('Return(%)=',round(out['Return [%]'],2))
print('Avg. Trade Duration=',out['Avg. Trade Duration'])
print('SQN=',round(out['SQN'],2))

bt.plot()

Equity Final($)= 1171.76
Trades= 35
Win Rate(%)= 91.43
Return(%)= 17.18
Avg. Trade Duration= 5 days 17:00:00
SQN= 5.33


In [6]:
print(out)

Start                     2018-02-03 03:00:00
End                       2020-02-12 07:00:00
Duration                    739 days 04:00:00
Exposure [%]                          85.7046
Equity Final [$]                      1363.79
Equity Peak [$]                       1365.27
Return [%]                            36.3795
Buy & Hold Return [%]                 0.35842
Max. Drawdown [%]                    -4.52364
Avg. Drawdown [%]                   -0.325152
Max. Drawdown Duration       69 days 08:00:00
Avg. Drawdown Duration        4 days 08:00:00
# Trades                                  109
Win Rate [%]                          83.4862
Best Trade [%]                        1.82767
Worst Trade [%]                       -2.0196
Avg. Trade [%]                       0.286808
Max. Trade Duration          24 days 04:00:00
Avg. Trade Duration           5 days 20:00:00
Expectancy [%]                       0.549093
SQN                                   4.59231
Sharpe Ratio                      

# 4. Optimisation<a id='optimisation'></a>

In [7]:
out2=bt.optimize(rci_s=range(9, 10),rci_m=range(21, 26),rci_l=range(52, 53))

In [8]:
print('▼最適化結果 ※',out2._trade_data.shape)
print('Equity Final($)=',round(out2['Equity Final [$]'],2))
print('Trades=',out2['# Trades'])
print('Win Rate(%)=',round(out2['Win Rate [%]'],2))
print('Return(%)=',round(out2['Return [%]'],2))
print('SQN=',round(out2['SQN'],2))
print('_strategy=',out2['_strategy'])

▼最適化結果 ※ (3150, 9)
Equity Final($)= 1363.79
Trades= 109
Win Rate(%)= 83.49
Return(%)= 36.38
SQN= 4.59
_strategy= RCIStrategy2(rci_s=9,rci_m=24,rci_l=52)


- Start・・・・・・・・・・・ヒストリカルデータの開始日時
- End・・・・・・・・・・・・ヒストリカルデータの終了日時
- Duration・・・・・・・・・ ヒストリカルデータの期間
- Exposure (%) ・・・・・・・ポジションを持っていた期間の割合（ポジションを持っていた期間÷全期間×100）
- Equity Final・・・・・・・ 最終金額
- Equity Peak・・・・・・・・最高金額
- Return (%)・・・・・・・・ 利益率=損益÷開始時所持金×100
- Buy & Hold Return (%)・・・（（終了時の終値 - 開始時の終値）÷ 開始時の終値）の絶対値×100
- Max. Drawdown (%)・・・・・最大下落率
- Avg. Drawdown (%)・・・・・平均下落率
- Max. Drawdown Duration・・ 最大下落期間
- Avg. Drawdown Duration・・ 平均下落期間
- Trades・・・・・・・・・・ 取引回数
- Win Rate (%)・・・・・・・ 勝率=勝ち取引回数÷全取引回数×100
- Best Trade (%)・・・・・・ 1回の取引での利益の最大値÷所持金×100
- Worst Trade (%)・・・・・・1回の取引での損失の最大値÷所持金×100
- Avg. Trade (%)・・・・・・ 損益の平均値÷所持金×100
- Max. Trade Duration・・・・1回の取引での最長期間
- Avg. Trade Duration・・・・1回の取引での平均期間
- Expectancy (%)・・・・・・ 期待値=平均利益×勝率＋平均損失×敗率（１取引で期待できる利益、正は資産が増え、負は資産が減る。）
- SQN(取引システム評価)・・2.0～：平均、2.5～：良い、3.0～：素晴らしい、5.1～：最高　※取引数が30以上で、SQN値は信頼できる。
- Sharpe Ratio・・・・・・・標準偏差に対するリターンの比率 ※シャープレシオとは利益とリスクの比率のことで、値が大きいほど資産曲線がなめらかになり安定性のある利益が見込めます。
- Sortino Ratio・・・・・・・下方リスクに対するリターンの比率 ※シャープレシオだけでは分からない下方リスクの抑制度合いを判断する場合に使われます。通常、この数値が大きいほど優れている（下落局面に強い）ことを示します。
- Calmar Ratio・・・・・・・最大損失率に対する年間平均収益の比率 ※値が低いほど指定された期間に渡ってリスク調整ベースで実行された投資は悪化し、値が高いほどパフォーマンスが向上します。

- 最上部のEquityのグラフは資金推移を表します。
- 2段目のProfit / lossはトレードロジックで行われたトレードを利益と損益で可視化します。
- 最下部は実際の終値のレートと単純移動平均（短期と長期）、さらにトレードも併せて可視化します。

In [9]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 5000)

xls_df = df_3150

# RCI列を追加
xls_df['RCI_s'] = RCI(xls_df.Close,9)
xls_df['RCI_m'] = RCI(xls_df.Close,24)
xls_df['RCI_l'] = RCI(xls_df.Close,52)

# マージして出力
df_concat = pd.concat([out._trade_data.tail(3150), xls_df], axis=1)
df_concat.to_excel("./Backtest_trade_data.xlsx",startrow=0, startcol=0)
display(df_concat.tail(5))

,Equity,Exit Entry,Exit Position,Entry Price,Exit Price,P/L,Returns,Drawdown,Drawdown Duration,Open,High,Low,Close,RCI_s,RCI_m,RCI_l
2020-02-11 15:00:00,1363.038415,NaN,NaN,NaN,NaN,NaN,NaN,0.001635,NaT,109.901,109.953,109.838,109.872,54.0,-23.0,77.0
2020-02-11 19:00:00,1364.067769,NaN,NaN,NaN,NaN,NaN,NaN,0.000881,NaT,109.873,109.899,109.757,109.789,67.0,-32.0,77.0
2020-02-11 23:00:00,1363.670910,NaN,NaN,NaN,NaN,NaN,NaN,0.001172,NaT,109.788,109.973,109.755,109.821,68.0,-43.0,77.0
2020-02-12 03:00:00,1363.980956,NaN,NaN,NaN,NaN,NaN,NaN,0.000945,NaT,109.820,109.844,109.734,109.796,50.0,-43.0,76.0
2020-02-12 07:00:00,1363.794928,NaN,NaN,NaN,NaN,NaN,NaN,0.001081,NaT,109.793,109.838,109.783,109.811,23.0,-41.0,75.0
